# 11. YÜZ ÖZELLİKLERİNİ KULLANMA (USİNG FACE FEATURE) 1.ADIM 12.YÜZ ÖZELLİKLERİNİ KULLANMA 2.ADIM VE 13. YÜZ ÖZELLİKLERİNİ KULLANMA 3. ADIM

In [2]:
import cv2
import numpy as np
import math

def findMaxContour(contours):
    max_i = 0
    max_area = 0
    
    for i in range(len(contours)):
        area_face = cv2.contourArea(contours[i])
        if max_area < area_face:
            max_area = area_face
            max_i = i
            
        try:
            c = contours[max_i]
            
        except:
            contours = [0]
            c = contours[0]
            
        return c
        
    
cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    if ret is False:
        break
    
    roi = frame[50:250,200:400] #frame[y1:y2,x1:x2] şeklindedir
    cv2.rectangle(frame,(200,50),(400,250),(0,0,250),0) # cv2.rectangle içine 4 değer alır frame,(x1,y1),(x2,y2),(renk değeri),kalınlık değeri dir bu değerler
                                                        # bu rectangle ın kalınlık değeri 0 olmak zorundadır çünkü kalınlığı artınca threshold'a maruz kalıp maskelenebilir
    hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    lower_color = np.array([0,45,79],dtype=np.uint8)
    upper_color = np.array([17,255,255],dtype=np.uint8)
    
    
    mask = cv2.inRange(hsv,lower_color,upper_color)
    
    kernel = np.ones((3,3),dtype=np.uint8)
    mask = cv2.dilate(mask,kernel,iterations=1)
    mask = cv2.medianBlur(mask,15)
    
    contours,ret = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        
            c = findMaxContour(contours)
            
            ensag = tuple(c[c[:, :, 0].argmin()][0])  # bu ifade argmin() yardımıyla bize tüm contour değerlerini dolanarak en küçüklerini bulacak
            ensol = tuple(c[c[:, :, 0].argmax()][0])
            entepe = tuple(c[c[:, :, 1].argmin()][0])  # tepe noktası y ekseninde olduğu için contour değerlerindeki 2. indisi aldık
            endip = tuple(c[c[:, :, 1].argmax()][0])
            
            cv2.circle(roi,ensag,5,(0,255,0),2)
            cv2.circle(roi,ensol,5,(0,255,0),2)
            cv2.circle(roi,entepe,5,(0,255,0),2)
            cv2.circle(roi,endip,5,(0,255,0),2)
            
            cv2.line(roi,ensol,entepe,(255,0,0),2)
            cv2.line(roi,entepe,ensag,(255,0,0),2)
            cv2.line(roi,ensag,endip,(255,0,0),2)
            cv2.line(roi,endip,ensol,(255,0,0),2)
            
            a = math.sqrt((ensag[0]-entepe[0])**2+(ensag[1]-entepe[1])**2)
            b = math.sqrt((endip[0]-ensag[0])**2+(endip[1]-ensag[1])**2)
            c = math.sqrt((endip[0]-entepe[0])**2+(endip[1]-entepe[1])**2)
            
            try:
                angle_ab = int(math.acos((a**2+b**2-c**2)/(2*b*c))*57)
                cv2.putText(roi,str(angle_ab),(ensag[0]-100+50,ensag[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
            except:
                cv2.putText(roi,"?",(ensag[0]-100+50,ensag[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
                
    cv2.imshow("frame",frame)
    cv2.imshow("roi",roi)
    cv2.imshow("mask",mask)
    
    if cv2.waitKey(5) == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()